In [1]:
import sys
sys.path.append('/home/alan/AlanLiang/Projects/AlanLiang/LiDAR-Layout')
from lidm.data.nuscenes_cube_dataset import NUSC_CUBE_DATASET
from scripts.vis_tools.functions.lidm_cond_sample import AE_Sampler
import polyscope as ps

To install torchsparse 1.4.0, please refer to https://github.com/mit-han-lab/torchsparse/tree/74099d10a51c71c14318bce63d6421f698b24f24
To install torchsparse 1.4.0, please refer to https://github.com/mit-han-lab/torchsparse/tree/74099d10a51c71c14318bce63d6421f698b24f24
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
sys.path.append('/home/alan/AlanLiang/Projects/AlanLiang/LiDAR-Layout/scripts')
# ckpt
ae_sampler = AE_Sampler(ckpt_path='/home/alan/AlanLiang/Projects/AlanLiang/LiDAR-Layout/models/ours/nuscenes/refine/voxel_ae/last.ckpt')
ae_sampler.build_model()
# dataset
ae_sampler.data_config.update({
    'split': 'val',
    'data_root': '/home/alan/AlanLiang/Dataset/Nuscenes'
})
dataset = NUSC_CUBE_DATASET(**ae_sampler.data_config)

Loading model from /home/alan/AlanLiang/Projects/AlanLiang/LiDAR-Layout/models/ours/nuscenes/refine/voxel_ae/last.ckpt


2025-04-11 15:28:51.999 | INFO     | lidm.modules.xcube.sunet:__init__:240 - latent dim: 8


In [3]:
def get_data_dict(index):
    data_dict = dataset.collate_fn([dataset.__getitem__(index)])
    for key, value in data_dict.items():
        data_dict[key] = value.to('cuda')
    return data_dict

In [4]:
index = 0
data_dict = get_data_dict(index)
logs = ae_sampler.model.log_images(data_dict)

In [5]:
for key, value in logs.items():
    print(key)
    print(value.shape)
    print('\n')

gt_xyz
(11671, 3)


pred_xyz
(10235, 3)




In [8]:
raw_points = data_dict['points_for_cube'][:,1:].detach().cpu().numpy()
ps.init()
ps.set_up_dir("z_up")
ps.set_ground_plane_mode("none")
# coarse stage
pred_xyz = logs["pred_xyz"]
ps.register_point_cloud(f"Pred Point", pred_xyz)
# ps.register_point_cloud(f"GT Point", raw_points)
ps.show()

In [7]:
ps.init()
ps.set_up_dir("z_up")
ps.set_ground_plane_mode("none")
gt_xyz = logs["gt_xyz"]
ps.register_point_cloud(f"GT Point", gt_xyz)
ps.show()

In [9]:
data_dict['points_for_cube'][:,1:].shape

torch.Size([34277, 3])